In [10]:
import networkx as nx
import numpy as np
from pgmpy.models.BayesianModel import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.factors.discrete import State
import random as random
import pandas as pd

In [2]:
student = BayesianModel([('diff', 'grade'), ('intel', 'grade')])
cpd_d = TabularCPD('diff', 2, [[0.6], [0.4]])
cpd_i = TabularCPD('intel', 2, [[0.7], [0.3]])
cpd_g = TabularCPD('grade', 3, [[0.3, 0.05, 0.9, 0.5], 
                                [0.4, 0.25,0.08, 0.3], 
                                [0.3, 0.7, 0.02, 0.2]],
               ['intel', 'diff'], [2, 2])

student.add_cpds(cpd_d, cpd_i, cpd_g)

In [3]:

def get_value_from_cpd(cpd_values,random_value):
    cpd_value_line = []
    current = 0;
    for i,cpd_value in enumerate(cpd_values):
        current += cpd_value
        if random_value<= current:
            return i

In [4]:
def get_value_from_cpd_evidence(cpd_values,evidence,random_value):
    k_length = len(cpd_values)
    cpd_value_line = []
    current = 0;
    for k in range(k_length):
        v = cpd_values[k][evidence[0]][evidence[1]]
        current += v
        if random_value<= current:
            return k
        #print(v)

### Prior Sampling

In [14]:
def prior_sampling(model,sample_size):
    
    sorted_order = list(nx.topological_sort(model))
    variable_count = len(model.nodes)
    size = sample_size * variable_count
    #print(size)
    r = [random.random() for _ in range(size)]
    k = 0
    temp = dict()
    sampled = []
    
    for i in range(sample_size):
        for node in sorted_order:
            cpd = model.get_cpds(node)
            evidence = cpd.variables[:0:-1]
            #print(evidence)  
            v = None
            if evidence:
                #
                evidence_value = []
                for ev in evidence:
                    evidence_value.append(temp[ev])   
                #v = get_value_from_cpd_evidence(cpd_g.values,evidence_value,r[k])
                v = get_value_from_cpd_evidence(model.values,evidence_value,r[k])
                
            else:
                #no evidence case
                v = get_value_from_cpd(cpd.values,r[k])
            
            temp[cpd.variables[0]] = v
            k+=1
            
        sampled.append(temp.copy())
        print(temp)
    return sampled

In [6]:
samples = prior_sampling(student,10)

{'intel': 0, 'diff': 0, 'grade': 2}
{'intel': 1, 'diff': 0, 'grade': 2}
{'intel': 1, 'diff': 0, 'grade': 1}
{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 0, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 0, 'grade': 1}
{'intel': 0, 'diff': 0, 'grade': 0}
{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 1, 'diff': 1, 'grade': 0}
{'intel': 0, 'diff': 0, 'grade': 0}


In [13]:
samples_df = pd.DataFrame(samples,columns=['diff','intel','grade'])
samples_df

,diff,intel,grade
0,0,0,2
1,0,1,2
2,0,1,1
3,1,0,0
4,0,0,2
5,0,0,1
6,0,0,0
7,1,0,0
8,1,1,0
9,0,0,0


### Likelihood Weighted